In [1]:
from datasets import load_dataset, load_metric


In [2]:
dataset = load_dataset("europa_eac_tm", language_pair=("pl", "en"))

Using custom data configuration pl2en-0da2ec5e9ea613fc
Reusing dataset europa_eac_tm (/home/bartek/.cache/huggingface/datasets/europa_eac_tm/pl2en-0da2ec5e9ea613fc/0.0.0/955b2501a836c2ea49cfe3e719aec65dcbbc3356bbbe53cf46f08406eb77386a)


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation', 'sentence_type'],
        num_rows: 4027
    })
})

In [4]:
metric = load_metric("sacrebleu")

In [5]:
dataset['train'][2]

{'translation': {'en': "The grant application will be processed by computer. All personal data (such as names, addresses, CVs, etc.) will be processed in accordance with Regulation (EC) No 45/2001 of the European Parliament and of the Council of 18 December 2000 on the protection of individuals with regard to the processing of personal data by the Community institutions and bodies and on the free movement of such data. Information provided by the applicants necessary in order to assess their grant application will be processed solely for that purpose by the department responsible for the programme concerned. On the applicant's request, personal data may be sent to the applicant to be corrected or completed. Any question relating to these data, should be addressed to the appropriate Agency to which the form must be submitted. Beneficiaries may lodge a complaint against the processing of their personal data with the European Data Protection Supervisor at anytime.",
  'pl': 'Wniosek o dof

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [7]:
model_name = "Helsinki-NLP/opus-mt-pl-en"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
import os

In [10]:
if not os.path.exists('model_checkpoints/base_model/'):
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
else:
    model = AutoModelForSeq2SeqLM.from_pretrained('model_checkpoints/base_model/')

In [11]:
model.save_pretrained('model_checkpoints/base_model')

In [110]:
# sample_input = dataset['train'][2]['translation']['pl']
# sample_output = dataset['train'][2]['translation']['en']
sample_input = "Jak masz na imię?"
sample_output = "What's your name?"
print(sample_input)
print(sample_output)

Jak masz na imię?
What's your name?


In [104]:
tokenizer.encode_plus(sample_input)

{'input_ids': [295, 669, 22, 2552, 7, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [105]:
print(tokenizer.encode_plus(sample_output))
with tokenizer.as_target_tokenizer():
    print(tokenizer.encode_plus(sample_output))

{'input_ids': [39, 6139, 6, 9, 15, 23, 1943, 22, 1326, 7, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [112, 6, 9, 79, 653, 7, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


In [106]:
tokenizer.vocab_size

63430

In [107]:
import torch

In [111]:
sample_input = tokenizer(sample_input, return_tensors='pt')

In [117]:
output = model.generate(**sample_input)

In [118]:
output

tensor([[63429,   112,     6,     9,    79,   653,     7,     0]])

In [119]:
output = output.detach().cpu().numpy()[0]

In [120]:
output

array([63429,   112,     6,     9,    79,   653,     7,     0])

In [124]:
output = tokenizer.decode(output, skip_special_tokens=True)

In [125]:
sample_output

"What's your name?"

In [126]:
output == sample_output

True